In [ ]:
import os
import pandas as pd
from tensorflow.python.summary.summary_iterator import summary_iterator

# Path to the TensorBoard log file
log_dir = "./model/ACDC/Cross_Pseudo_Supervision_3/unet/log"  # Replace with your log directory
output_excel = "tensorboard_logs.xlsx"

# Function to parse TensorBoard logs
def parse_events(log_dir):
    data = []
    for file in os.listdir(log_dir):
        if file.startswith("events.out"):
            log_file = os.path.join(log_dir, file)
            for event in summary_iterator(log_file):
                for value in event.summary.value:
                    if hasattr(value, "simple_value"):  # Skip non-scalar values
                        data.append({
                            "step": event.step,
                            "wall_time": event.wall_time,
                            "tag": value.tag,
                            "value": value.simple_value
                        })
    return pd.DataFrame(data)

# Convert logs to DataFrame
df = parse_events(log_dir)

# Save DataFrame to Excel
if not df.empty:
    df.to_excel(output_excel, index=False, engine='openpyxl')
    print(f"TensorBoard logs saved to {output_excel}")
else:
    print("No data found in TensorBoard logs.")


TensorBoard logs saved to tensorboard_logs.xlsx


In [7]:
import os
import pandas as pd
from collections import defaultdict
from tensorflow.python.summary.summary_iterator import summary_iterator

# Path to the TensorBoard log file
log_dir = "./model/ACDC/Cross_Pseudo_Supervision_3/unet/log"  # Replace with your log directory
output_excel = "tensorboard_logs_with_columns.xlsx"

# Function to parse TensorBoard logs and organize data
def parse_events(log_dir):
    step_data = defaultdict(dict)
    for file in os.listdir(log_dir):
        if file.startswith("events.out"):
            log_file = os.path.join(log_dir, file)
            for event in summary_iterator(log_file):
                step = event.step
                for value in event.summary.value:
                    if hasattr(value, "simple_value"):  # Skip non-scalar values
                        step_data[step]["wall_time"] = event.wall_time
                        step_data[step][value.tag] = value.simple_value

    # Convert the nested dictionary to a DataFrame
    df = pd.DataFrame.from_dict(step_data, orient="index")
    df.index.name = "step"
    return df.reset_index()

# Convert logs to DataFrame
df = parse_events(log_dir)

# Save DataFrame to Excel
if not df.empty:
    df.to_excel(output_excel, index=False, engine='openpyxl')
    print(f"TensorBoard logs saved to {output_excel}")
else:
    print("No data found in TensorBoard logs.")


TensorBoard logs saved to tensorboard_logs_with_columns.xlsx


In [8]:
import os
import pandas as pd
from collections import defaultdict
from tensorflow.python.summary.summary_iterator import summary_iterator

# Path to the TensorBoard log directory
log_dir = "./model/ACDC/Cross_Pseudo_Supervision_3/unet/log"  # Replace with your log directory
output_excel = "latest_tensorboard_logs.xlsx"

# Function to find the latest log file
def get_latest_log_file(log_dir):
    files = [
        os.path.join(log_dir, f)
        for f in os.listdir(log_dir)
        if f.startswith("events.out")
    ]
    if not files:
        raise FileNotFoundError("No TensorBoard log files found in the directory.")
    # Get the latest file based on creation or modification time
    latest_file = max(files, key=os.path.getmtime)
    return latest_file

# Function to parse the latest TensorBoard log file
def parse_latest_log_file(log_file):
    step_data = defaultdict(dict)
    for event in summary_iterator(log_file):
        step = event.step
        for value in event.summary.value:
            if hasattr(value, "simple_value"):  # Skip non-scalar values
                step_data[step]["wall_time"] = event.wall_time
                step_data[step][value.tag] = value.simple_value

    # Convert the nested dictionary to a DataFrame
    df = pd.DataFrame.from_dict(step_data, orient="index")
    df.index.name = "step"
    return df.reset_index()

# Main execution
try:
    latest_log_file = get_latest_log_file(log_dir)
    print(f"Processing latest log file: {latest_log_file}")
    df = parse_latest_log_file(latest_log_file)

    # Save DataFrame to Excel
    if not df.empty:
        df.to_excel(output_excel, index=False, engine='openpyxl')
        print(f"Latest TensorBoard log saved to {output_excel}")
    else:
        print("No data found in the latest TensorBoard log file.")
except FileNotFoundError as e:
    print(e)


Processing latest log file: ./model/ACDC/Cross_Pseudo_Supervision_3/unet/log/events.out.tfevents.1735984591.treerspeaking-pc
Latest TensorBoard log saved to latest_tensorboard_logs.xlsx


In [12]:
import os
import pandas as pd
from collections import defaultdict
from tensorflow.python.summary.summary_iterator import summary_iterator

def convert_latest_tensorboard_log_to_excel(log_dir, output_excel):
    """
    Convert the latest TensorBoard log file in the specified directory to an Excel sheet.

    Parameters:
        log_dir (str): Path to the directory containing TensorBoard log files.
        output_excel (str): Path to save the resulting Excel file.
    """
    def get_latest_log_file(log_dir):
        """
        Identify the most recently modified TensorBoard log file in the directory.

        Parameters:
            log_dir (str): Directory containing TensorBoard log files.

        Returns:
            str: Path to the latest TensorBoard log file.

        Raises:
            FileNotFoundError: If no TensorBoard log files are found.
        """
        files = [
            os.path.join(log_dir, f)
            for f in os.listdir(log_dir)
            if f.startswith("events.out")
        ]
        if not files:
            raise FileNotFoundError("No TensorBoard log files found in the directory.")
        return max(files, key=os.path.getmtime)

    def parse_log_file(log_file):
        """
        Parse the TensorBoard log file and extract its data.

        Parameters:
            log_file (str): Path to the TensorBoard log file.

        Returns:
            pd.DataFrame: DataFrame containing the parsed data.
        """
        step_data = defaultdict(dict)
        for event in summary_iterator(log_file):
            step = event.step
            for value in event.summary.value:
                if hasattr(value, "simple_value"):  # Skip non-scalar values
                    step_data[step]["wall_time"] = event.wall_time
                    step_data[step][value.tag] = value.simple_value

        # Convert the nested dictionary to a DataFrame
        df = pd.DataFrame.from_dict(step_data, orient="index")
        df.index.name = "step"
        return df.reset_index()

    try:
        # Get the latest log file
        latest_log_file = get_latest_log_file(log_dir)
        print(f"Processing latest log file: {latest_log_file}")

        # Parse the log file
        df = parse_log_file(latest_log_file)

        # Save DataFrame to Excel
        if not df.empty:
            df.to_excel(output_excel, index=False, engine='openpyxl')
            print(f"Latest TensorBoard log saved to {output_excel}")
        else:
            print("No data found in the latest TensorBoard log file.")
    except FileNotFoundError as e:
        print(e)

# # Example usage
# log_dir = "./model/ACDC/Cross_Pseudo_Supervision_3/unet/log"  # Replace with your log directory
# output_excel = "latest_tensorboard_123logs.xlsx"  # Replace with desired output file path
convert_latest_tensorboard_log_to_excel("./model/ACDC/Cross_Pseudo_Supervision_3/unet/log", "latest_cps_unet_log.xlsx")
convert_latest_tensorboard_log_to_excel("./model/ACDC/Uncertainty_Aware_Mean_Teacher_3_labeled/unet/log", "latest_UAMT_unet_log.xlsx")
convert_latest_tensorboard_log_to_excel("./model/ACDC/Mean_Teacher_3_labeled/unet_cct/log", "latest_cct_unet_log.xlsx")
convert_latest_tensorboard_log_to_excel("./model/ACDC/Mean_Teacher_3_labeled/unet_urpc/log", "latest_urpc_unet_log.xlsx")
convert_latest_tensorboard_log_to_excel("./model/ACDC/Mean_Teacher_3_labeled/unet/log", "latest_MT_unet_log.xlsx")


Processing latest log file: ./model/ACDC/Cross_Pseudo_Supervision_3/unet/log/events.out.tfevents.1735984591.treerspeaking-pc
Latest TensorBoard log saved to latest_cps_unet_log.xlsx
Processing latest log file: ./model/ACDC/Uncertainty_Aware_Mean_Teacher_3_labeled/unet/log/events.out.tfevents.1735977032.treerspeaking-pc
Latest TensorBoard log saved to latest_UAMT_unet_log.xlsx
Processing latest log file: ./model/ACDC/Mean_Teacher_3_labeled/unet_cct/log/events.out.tfevents.1735946212.treerspeaking-pc
Latest TensorBoard log saved to latest_cct_unet_log.xlsx
Processing latest log file: ./model/ACDC/Mean_Teacher_3_labeled/unet_urpc/log/events.out.tfevents.1735934344.treerspeaking-pc
Latest TensorBoard log saved to latest_urpc_unet_log.xlsx
Processing latest log file: ./model/ACDC/Mean_Teacher_3_labeled/unet/log/events.out.tfevents.1735972053.treerspeaking-pc
Latest TensorBoard log saved to latest_MT_unet_log.xlsx
